# Phase I : Data Extraction

This Phase Involves Extracting Data From The Resumes. Optical Character Recognition (OCR) Tools Can Extract Text From Resume Images. Then The Text Is Pre-Processed To Extract The Relevant Information Such As Name, Email, Phone Number, Education Details, Work Experience, Skills, Etc.I

In [34]:
#from PIL import Image
import pytesseract
import cv2
import spacy
from spacy.matcher import Matcher
from rake_nltk import Rake
import os
import docx
import PyPDF2
import numpy as np 
import pandas as pd 


import sys
import nltk
import re
import requests
import subprocess
import pandas as pd
from tabulate import tabulate
#from pdfminer.high_level import extract_text
#from PyPDF2 import PdfReader, PdfFileWriter, PdfFileMerger


# Load Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Update with your Tesseract installation path

nlp = spacy.load('en_core_web_sm')

from nanonets import NANONETSOCR

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#extract_text_from_pdf: This function takes a PDF or  file path as input and extracts text from all the pages in the PDF using NANONETSOCR

def extract_text_from_pdf_or_image(pdf_path):
    model = NANONETSOCR()
    model.set_token('87dcd5e7-53e9-11ee-bb5e-c26e11187d6d')
    text = model.convert_to_string(pdf_path,formatting='lines') 
    return text


#extract_text_from_docx: This function takes a DOCX file path as input and extracts text from the document using the docx library.
def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    text = '\n'.join([para.text for para in doc.paragraphs])
    return text


In [3]:
def extract_contact_number_from_resume(text):
    contact_number = None

    # Use regex pattern to find a potential contact number
    pattern = r"\b(?:\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"
    match = re.search(pattern, text)
    if match:
        contact_number = match.group()

    return contact_number

def extract_email_from_resume(text):
    email = None

    # Use regex pattern to find a potential email address
    pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
    match = re.search(pattern, text)
    if match:
        email = match.group()

    return email

def extract_skills_from_resume(text, skills_list):
    skills = []

    for skill in skills_list:
        pattern = r"\b{}\b".format(re.escape(skill))
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            skills.append(skill)

    return skills

def extract_education_from_resume(text):
    education = []

    # Use regex pattern to find education information
    pattern = r"(?i)(?:Bsc|\bB\.\w+|\bM\.\w+|\bPh\.D\.\w+|\bBachelor(?:'s)?|\bMaster(?:'s)?|\bPh\.D)\s(?:\w+\s)*\w+"
    matches = re.findall(pattern, text)
    for match in matches:
        education.append(match.strip())

    return education

def extract_name_from_resume(resume_text):
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)

    # Define name patterns
    patterns = [
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name, Middle name, and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]  # First name, Middle name, Middle name, and Last name
        # Add more patterns as needed
    ]

    for pattern in patterns:
        matcher.add('NAME', patterns=[pattern])

    doc = nlp(resume_text)
    matches = matcher(doc)

    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text

    return None

# Function to perform Aptitude extraction using RAKE
def extract_Aptitude_frome_resume(text):
    r = Rake(min_length=2, max_length=4)  # Adjust min_length and max_length as needed
    r.extract_keywords_from_text(text)
    ranked_phrases = r.get_ranked_phrases()
    # Additional stop words specific to resumes that we want to exclude
    STOPWORDS = [
    'resume', 'cv', 'summary', 'experience', 'education', 'skill', 'skills', 'abilities', 'project', 'projects',
    'work', 'worked', 'company', 'companies', 'responsibilities', 'responsibility', 'achieved', 'team', 'teams']
    ap = [phrase for phrase in ranked_phrases if not any(stopword in phrase.lower() for stopword in STOPWORDS)]
    return ap



In [22]:
def parse_resume(resume_text,file_name):
    skills_list = ['Python', 'Data Analysis', 'Machine Learning', 'Communication', 'Project Management', 'Deep Learning', 'SQL', 'Tableau']
    
    name = extract_name_from_resume(resume_text)
    email = extract_email_from_resume(resume_text)
    phone = extract_contact_number_from_resume(resume_text)
    skills = extract_skills_from_resume(resume_text,skills_list)
    education = extract_education_from_resume(resume_text)
    Aptitude = extract_Aptitude_frome_resume(resume_text)
    
    resumes_ap = [] # List to store extracted skills from resumes
    if Aptitude:
        resumes_ap.append(' '.join(Aptitude))  # Combine skills as a single string

    
 # Replace empty lists with a single empty string
    name = name or ' '
    email = email or ' '
    phone = phone or ' '
    skills = skills or ' '
    education = education or ' '
    resumes_ap=resumes_ap or ' '
    info = {
        "File_name":file_name,
        "Name": name,
        "Email": email,
        "Phone": phone,
        "Skills": skills,
        "Education":education,
        "Resumes_ap":resumes_ap
    }
    return info



In [48]:
if __name__ == '__main__':
    # Directory containing candidate resumes (PDF, DOCX, and images)
    directory = 'C:/Users/user/Documents/GOMYCODE/ResumeParsing/dataset'
    AllresumeInfo=[]
    #resume_paths = [r"C:/Users/user/Documents/GOMYCODE/ResumeParsing/dataset/3.pdf"]
    #file_path =os.listdir(directory)
    #for filename in file_path:
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)        
        if os.path.isfile(file_path) and (file_path.lower().endswith('.pdf') or file_path.lower().endswith('.png') or file_path.lower().endswith('.jpg'))  :
            #resume_text = extract_text_from_pdf(file_path)
            resume_text=extract_text_from_pdf_or_image(file_path)
            info=parse_resume(resume_text,filename)
            AllresumeInfo.append(info)
        elif os.path.isfile(file_path) and file_path.lower().endswith('.docx'):
            resume_text = extract_text_from_docx(file_path)
            info=parse_resume(resume_text,filename)
            AllresumeInfo.append(info)    
        else:
            print('Unsupported File Format:', filename)
        continue
        

Unsupported File Format: 25.jfif
Unsupported File Format: 26.jfif
Unsupported File Format: 28.jpeg
Unsupported File Format: 29.jfif
Unsupported File Format: prog2.ipynb


In [49]:
# transformation en datafram
File_name=[]
Name =[]
Email =[]
Phone = []
Skills = []
Education = []
Resumes_ap=[]
for i in range(len(AllresumeInfo)):
        File_name.append(AllresumeInfo[i]['File_name'])
        Name.append(AllresumeInfo[i]['Name'])
        Email.append(AllresumeInfo[i]['Email'])
        Phone.append(AllresumeInfo[i]['Phone'])
        Skills.append(AllresumeInfo[i]['Skills'])
        Education.append(AllresumeInfo[i]['Education'])
        Resumes_ap.append(AllresumeInfo[i]['Resumes_ap'])
        
        
        
d = {
    'File_name':File_name, 
    'Name': Name, 
    'Email': Email, 
    'Phone':Phone,
    'Skills': Skills, 
    'Education':Education,
    "Resumes_ap":Resumes_ap 
    }
df = pd.DataFrame(data = d)
pd.set_option('display.max_row', 111)
pd.set_option('display.max_column', 111)
df.head()

,File_name,Name,Email,Phone,Skills,Education,Resumes_ap
0,1.pdf,Avishek Kumar,aviw.upes01@gmail.com,91999900011,"[Python, Data Analysis, Machine Learning, Deep...",[Ph.D Universiti\nAchievements\nTeknologi PETR...,[speed motion 1 increasing python based al sim...
1,10.pdf,Probabilistic Models,,,"[Python, Data Analysis, Machine Learning, Comm...",,[numpy pandas matplotlib etc al currency conve...
2,11.pdf,Fulkar Khan,fulkarkhan9980@gmail.com,8443405050,"[Python, Machine Learning, Deep Learning, SQL,...",,[providing actionable insights aligned develop...
3,12.pdf,Kunika Bhargav,bhargav496@gmail.com,7983924436,"[Python, Machine Learning, Deep Learning, SQL,...",[B.E in Electronics Engineering\nProiect focus...,[sql postgresql mysql banglore product recomme...
4,13.pdf,KAPIL GAWANDE,gkapil8019@gmail.com,9522330896,"[Python, Machine Learning, Deep Learning, SQL]",[Bachelor of Engineering],[gkapil8019 https :/ lwww datapre processing m...


# Phase II: Data Cleaning
In This Phase, The Extracted Data Is Cleaned And Normalized By Removing Noise, Formatting Inconsistencies, And Irrelevant Information.

In [50]:
tab=df.copy()

In [51]:
df.head(25).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
File_name,1.pdf,10.pdf,11.pdf,12.pdf,13.pdf,14.pdf,15.pdf,16.pdf,17.pdf,18.pdf,19.pdf,2.pdf,20.pdf,21.docx,22.docx,23.docx,24.png,27.jpg,3.pdf,4.pdf,5.pdf,6.pdf,7.pdf,8.pdf,9.pdf
Name,Avishek Kumar,Probabilistic Models,Fulkar Khan,Kunika Bhargav,KAPIL GAWANDE,Machine Learning,Kvoti Khan,SAGAR KURUVA,Siddhi Shukla,Farukh Sharma,ABC KUMAR,ANMISHA MURARKA,CURRICULUM VITAE,ASJIhina Mahajan,Sunit Kaushik,Curriculum Vitaé,Diana Dawa,Gunjan Parmer,Aankit Sharma,Aashok Kumar,PSWATHI VIJAYAN,Bhubaneswar A,KANOJA KUMAR,Infosys System,A Data
Email,aviw.upes01@gmail.com,,fulkarkhan9980@gmail.com,bhargav496@gmail.com,gkapil8019@gmail.com,pvkanohara@gmail.com,,,Ca54@gmail.com,,sinha2010@gmail.con,anmishamurarka@gmail.con,chawlasurili@gmail.com,,,chandanalakhera35@gmail.com,diana@novoresume.com,,aankit9418@gmail.com,aashokrekumar@gmail.com,pswathinrp@gmail.com,,kanoi43@gmail.com,,Mohitifha.mi58@gmail.com
Phone,91999900011,,8443405050,7983924436,9522330896,11-7192698493,09028202799,91 1160601004,9591213356,91 8234540343,917734897148,9199422433,91-2382257665,,,6244330054,044 412 2020,,,91-0176884977,9089319891,91 1121661342,91 7978471213,,7987743912
Skills,"[Python, Data Analysis, Machine Learning, Deep...","[Python, Data Analysis, Machine Learning, Comm...","[Python, Machine Learning, Deep Learning, SQL,...","[Python, Machine Learning, Deep Learning, SQL,...","[Python, Machine Learning, Deep Learning, SQL]","[Python, Data Analysis, Machine Learning, Deep...","[Python, Machine Learning, Deep Learning, SQL,...","[Python, Machine Learning, Deep Learning, SQL,...","[Python, Machine Learning, Communication, SQL]","[Python, Data Analysis, Machine Learning, Comm...","[Python, Machine Learning, Communication, Deep...","[Python, Machine Learning, Tableau]","[Python, Data Analysis, Machine Learning, Deep...","[Python, Data Analysis, Machine Learning, Comm...",[Python],"[Communication, SQL, Tableau]","[Python, Machine Learning, Deep Learning, SQL,...",[Communication],"[Python, Data Analysis, Machine Learning, SQL,...","[Python, Data Analysis, Machine Learning, Deep...","[Python, Machine Learning, Deep Learning]","[Python, Data Analysis, Machine Learning, Deep...","[Python, Data Analysis, Machine Learning, Comm...","[Python, Data Analysis, SQL]","[Python, Data Analysis, Machine Learning, Deep..."
Education,[Ph.D Universiti\nAchievements\nTeknologi PETR...,,,[B.E in Electronics Engineering\nProiect focus...,[Bachelor of Engineering],,[M.Tech 2016 EECE YMCA UST Faridabad 81],,[master driver tables that have member level i...,[Bachelor of Technology],[Master Tool and Cluster ECU software developm...,,[Bachelor of Chitkara School Of Chitkara Unive...,"[Bachelor of Science in Physics, Master of Sci...",,,[Master of Science in Computer Science and\nan...,,,[Bachelor of Technology],"[BACHELOR OF TECHNOLOGY\nBuild using Python, B...",[Bachelor of Technology],[Bachelor of Technology],,
Resumes_ap,[speed motion 1 increasing python based al sim...,[numpy pandas matplotlib etc al currency conve...,[providing actionable insights aligned develop...,[sql postgresql mysql banglore product recomme...,[gkapil8019 https :/ lwww datapre processing m...,[web scraping statistical modelling system rec...,[type ii fuzzy set 4 12th 2009 non prof rekha ...,[solve challenging business problems software ...,[xgboost feature importance 5 changing target ...,[tested multimodel algorithms based pandas mat...,[world problems pro ciency useful life beansta...,[facebook comment volume prediction candidates...,[roles like admin bidder role revolved around ...,[solved many question papers national conferen...,[implement restful api ’ restful api ’ create ...,[sitabpur dabral colony kotdwar reading travel...,[test automation python scikit scripting langu...,[mass communication ba 2016 blgd contert ahout...,[means clustering classification svim time ser...,[time loan repayment 1 professional developmen...,[solve real world pro

In [52]:
# nomber of lines and columns
df.shape

(25, 7)

In [53]:
df.columns.values

array(['File_name', 'Name', 'Email', 'Phone', 'Skills', 'Education',
       'Resumes_ap'], dtype=object)

In [54]:
# Drop File_name Column as it is seems to be an unique identifier for each row
df.drop(["File_name"],axis=1,inplace=True)

In [55]:
df["Name"]

0            Avishek Kumar
1     Probabilistic Models
2              Fulkar Khan
3           Kunika Bhargav
4            KAPIL GAWANDE
5         Machine Learning
6               Kvoti Khan
7             SAGAR KURUVA
8            Siddhi Shukla
9            Farukh Sharma
10               ABC KUMAR
11         ANMISHA MURARKA
12        CURRICULUM VITAE
13        ASJIhina Mahajan
14           Sunit Kaushik
15        Curriculum Vitaé
16              Diana Dawa
17           Gunjan Parmer
18           Aankit Sharma
19            Aashok Kumar
20         PSWATHI VIJAYAN
21           Bhubaneswar A
22            KANOJA KUMAR
23          Infosys System
24                  A Data
Name: Name, dtype: object

In [56]:
df["Name"].replace(to_replace = {'A Data':' ','Infosys System':' ','Curriculum Vitaé':' ','CURRICULUM VITAE':' ','Machine Learning':' ','Probabilistic Models':' '},inplace=True)

In [58]:
df["Name"]

0        Avishek Kumar
1                     
2          Fulkar Khan
3       Kunika Bhargav
4        KAPIL GAWANDE
5                     
6           Kvoti Khan
7         SAGAR KURUVA
8        Siddhi Shukla
9        Farukh Sharma
10           ABC KUMAR
11     ANMISHA MURARKA
12                    
13    ASJIhina Mahajan
14       Sunit Kaushik
15                    
16          Diana Dawa
17       Gunjan Parmer
18       Aankit Sharma
19        Aashok Kumar
20     PSWATHI VIJAYAN
21       Bhubaneswar A
22        KANOJA KUMAR
23                    
24                    
Name: Name, dtype: object

In [59]:
df['Email']

0           aviw.upes01@gmail.com
1                                
2        fulkarkhan9980@gmail.com
3            bhargav496@gmail.com
4            gkapil8019@gmail.com
5            pvkanohara@gmail.com
6                                
7                                
8                  Ca54@gmail.com
9                                
10            sinha2010@gmail.con
11       anmishamurarka@gmail.con
12         chawlasurili@gmail.com
13                               
14                               
15    chandanalakhera35@gmail.com
16           diana@novoresume.com
17                               
18           aankit9418@gmail.com
19        aashokrekumar@gmail.com
20           pswathinrp@gmail.com
21                               
22              kanoi43@gmail.com
23                               
24       Mohitifha.mi58@gmail.com
Name: Email, dtype: object

In [33]:
df["Phone"]

0       91999900011
1                  
2        8443405050
3        7983924436
4        9522330896
5     11-7192698493
6       09028202799
7     91 1160601004
8        9591213356
9     91 8234540343
10     917734897148
11       9199422433
12    91-2382257665
13                 
14                 
15       6244330054
16     044 412 2020
17                 
18                 
19    91-0176884977
20       9089319891
21    91 1121661342
22    91 7978471213
23                 
24       7987743912
Name: Phone, dtype: object

In [60]:
#transformer une liste en string 
def transformer_liste_string(df,col):
    for i in range(len(df[col])):
        df[col][i]=', '.join(df[col][i])


In [61]:
transformer_liste_string(df,'Skills')
transformer_liste_string(df,'Education')
transformer_liste_string(df,'Resumes_ap')

In [64]:
df['Resumes_ap'] 
#'Skills', 'Education','Resumes_ap'

0     speed motion 1 increasing python based al simu...
1     numpy pandas matplotlib etc al currency conver...
2     providing actionable insights aligned develop ...
3     sql postgresql mysql banglore product recommen...
4     gkapil8019 https :/ lwww datapre processing ma...
5     web scraping statistical modelling system reco...
6     type ii fuzzy set 4 12th 2009 non prof rekha v...
7     solve challenging business problems software e...
8     xgboost feature importance 5 changing target d...
9     tested multimodel algorithms based pandas matp...
10    world problems pro ciency useful life beanstal...
11    facebook comment volume prediction candidates ...
12    roles like admin bidder role revolved around u...
13    solved many question papers national conferenc...
14    implement restful api ’ restful api ’ create a...
15    sitabpur dabral colony kotdwar reading travell...
16    test automation python scikit scripting langua...
17    mass communication ba 2016 blgd contert ah

In [65]:
def preprocess(txt):
    # convert all characters in the string to lower case
    txt = txt.lower()
    # remove non-english characters, punctuation and numbers
    txt = re.sub('[^a-zA-Z]', ' ', txt)
    txt = re.sub('http\S+\s*', ' ', txt)  # remove URLs
    txt = re.sub('RT|cc', ' ', txt)  # remove RT and cc
    txt = re.sub('#\S+', '', txt)  # remove hashtags
    txt = re.sub('@\S+', '  ', txt)  # remove mentions
    txt = re.sub('\s+', ' ', txt)  # remove extra whitespace
    # tokenize word
    txt = nltk.tokenize.word_tokenize(txt)
    # remove stop words
    txt = [w for w in txt if not w in nltk.corpus.stopwords.words('english')]
    

    return ' '.join(txt)

In [66]:
# preprocessing text
df['Skills'] = df['Skills'].apply(lambda w: preprocess(w))



('Education', 'Resumes_ap')

In [67]:
df['Education'] = df['Skills'].apply(lambda w: preprocess(w))
df['Resumes_ap'] = df['Skills'].apply(lambda w: preprocess(w))

In [68]:
df.head(25).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
Name,Avishek Kumar,,Fulkar Khan,Kunika Bhargav,KAPIL GAWANDE,,Kvoti Khan,SAGAR KURUVA,Siddhi Shukla,Farukh Sharma,ABC KUMAR,ANMISHA MURARKA,,ASJIhina Mahajan,Sunit Kaushik,,Diana Dawa,Gunjan Parmer,Aankit Sharma,Aashok Kumar,PSWATHI VIJAYAN,Bhubaneswar A,KANOJA KUMAR,,
Email,aviw.upes01@gmail.com,,fulkarkhan9980@gmail.com,bhargav496@gmail.com,gkapil8019@gmail.com,pvkanohara@gmail.com,,,Ca54@gmail.com,,sinha2010@gmail.con,anmishamurarka@gmail.con,chawlasurili@gmail.com,,,chandanalakhera35@gmail.com,diana@novoresume.com,,aankit9418@gmail.com,aashokrekumar@gmail.com,pswathinrp@gmail.com,,kanoi43@gmail.com,,Mohitifha.mi58@gmail.com
Phone,91999900011,,8443405050,7983924436,9522330896,11-7192698493,09028202799,91 1160601004,9591213356,91 8234540343,917734897148,9199422433,91-2382257665,,,6244330054,044 412 2020,,,91-0176884977,9089319891,91 1121661342,91 7978471213,,7987743912
Skills,python data analysis machine learning deep lea...,python data analysis machine learning communic...,python machine learning deep learning sql tableau,python machine learning deep learning sql tableau,python machine learning deep learning sql,python data analysis machine learning deep lea...,python machine learning deep learning sql tableau,python machine learning deep learning sql tableau,python machine learning communication sql,python data analysis machine learning communic...,python machine learning communication deep lea...,python machine learning tableau,python data analysis machine learning deep lea...,python data analysis machine learning communic...,python,communication sql tableau,python machine learning deep learning sql tableau,communication,python data analysis machine learning sql tableau,python data analysis machine learning deep lea...,python machine learning deep learning,python data analysis machine learning deep lea...,python data analysis machine learning communic...,python data analysis sql,python data analysis machine learning deep lea...
Education,python data analysis machine learning deep lea...,python data analysis machine learning communic...,python machine learning deep learning sql tableau,python machine learning deep learning sql tableau,python machine learning deep learning sql,python data analysis machine learning deep lea...,python machine learning deep learning sql tableau,python machine learning deep learning sql tableau,python machine learning communication sql,python data analysis machine learning communic...,python machine learning communication deep lea...,python machine learning tableau,python data analysis machine learning deep lea...,python data analysis machine learning communic...,python,communication sql tableau,python machine learning deep learning sql tableau,communication,python data analysis machine learning sql tableau,python data analysis machine learning deep lea...,python machine learning deep learning,python data analysis machine learning deep lea...,python data analysis machine learning communic...,python data analysis sql,python data analysis machine learning deep lea...
Resumes_ap,python data analysis machine learning deep lea...,python data analysis machine learning communic...,python machine learning deep learning sql tableau,python machine learning deep learning sql tableau,python machine learning deep learning sql,python data analysis machine learning deep lea...,python machine learning deep learning sql tableau,python machine learning deep learning sql tableau,python machine learning communication sql,python data analysis machine learning communic...,python machine learning communication deep lea...,python machine learning tableau,python data analysis machine learning deep lea...,python data analysis machine learning communic...,python,communication sql tableau,python machine learning deep learning sql tableau,communication,python data analysis machine learning sql tableau,python data analysis machine learning deep lea...,python 